In [1]:
import os

In [2]:
%pwd

'd:\\recommendation-engine\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\recommendation-engine'

In [5]:
import pandas as pd

In [6]:
data1 = pd.read_csv("artifacts/data_ingestion/generated_data/bookings.csv")
data2 = pd.read_csv("artifacts/data_ingestion/generated_data/comments.csv")
data3 = pd.read_csv("artifacts/data_ingestion/generated_data/likes.csv")
data1.head()
data2.head()
data3.head()

,user_id,liked_event_id,liked_event_organizer_id,like_id
0,c58d422b-c612-4088-8645-6b65e8de2992,f36541da-a645-40ea-8d0c-dd11452b36ef,545c1990-0afa-4916-9699-bd6201976dfe,7f7acffe-72cd-4342-9a76-b9121d1081fb
1,f4ba7042-6fae-4a7a-ab43-cd20bbcb0979,f3159360-6da4-42f4-99d7-34b078d194da,14257c8a-014a-4a69-a3ea-4bcc57a0396f,340e2498-0efb-4716-b1d3-448c40657b9c
2,5e093549-092a-4341-83a0-6a2164cc84f3,6b3f6145-572e-4494-87f5-5e5891635eff,b173f9c2-3825-4c87-a724-99c2882030e6,b8fb93c9-08cd-4403-817a-2b1650c3b936
3,ed762490-2c8a-4126-aae4-65927fd6c998,0f4b8b49-4acf-42a3-b76c-dd2452c52434,42409373-2ba7-4322-a0b5-1aa437afec57,6d639090-faad-4f44-9662-d3624a16d07f
4,63211825-4aa2-498f-b5df-a853ee0df8a8,0c8e254b-96bd-45a0-bfff-232a3ed50ba6,5a0e9844-ecb1-4ccf-8b50-a6bc118872f5,badb04e3-7c28-4f00-9784-0968fe10f12f


In [7]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 4 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   user_id                    150000 non-null  object
 1   booked_event_id            150000 non-null  object
 2   booked_event_organizer_id  150000 non-null  object
 3   booking_id                 150000 non-null  object
dtypes: object(4)
memory usage: 4.6+ MB


In [8]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    merged_data_url: Path
    unzip_data_dir: Path
    all_schema: dict

In [9]:
from src.hybrid_recommender.constants import *
from src.hybrid_recommender.utils.common import read_yaml, create_directories

In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir = config.unzip_data_dir,
            merged_data_url = config.merged_data_url,
            all_schema=schema,
        )

        return data_validation_config

In [11]:
import os
from src.hybrid_recommender import logger


In [12]:
class DataValiadtion:
    def __init__(self, config: DataValidationConfig):
        self.config = config


    def validate_all_columns(self)-> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.merged_data_url)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            
            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

            return validation_status
        
        except Exception as e:
            raise e

In [13]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValiadtion(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2025-06-30 16:37:13,991: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-06-30 16:37:13,996: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-30 16:37:13,996: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-06-30 16:37:13,996: INFO: common: created directory at: artifacts]
[2025-06-30 16:37:13,996: INFO: common: created directory at: artifacts/data_validation]
